In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import os, requests, json, geojson, gc
import pandas as pd
import geopandas as gpd
import ee
import geemap
import boto3

In [ ]:
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-indicators'
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"
boundary_ext = '/data/boundaries/'
indicators_file_aws = 'indicators/indicators.csv'

In [ ]:
OUTPUT_FILENAME = 'LND-5-habitat-types-restored.csv'

In [ ]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir + boundary_ext + 'boundary_georef.csv')
boundary_georef

In [ ]:
# Convert from GLAD ARD types to simpler habitat types
# ARD legend is at https://storage.googleapis.com/earthenginepartners-hansen/GLCLU2000-2020/legend.xlsx
LANDCLASSES = {}
for j in range(0, 19):
    LANDCLASSES[j] = {'name': 'upland sparse vegetation', 'classval': 1, 'is_habitat': False}
for j in range(19, 25):
    LANDCLASSES[j] = {'name': 'upland short vegetation', 'classval': 2, 'is_habitat': True}
for j in range(25, 49):
    LANDCLASSES[j] = {'name': 'upland tree cover', 'classval': 3, 'is_habitat': True}
for j in range(100, 119):
    LANDCLASSES[j] = {'name': 'wetland sparse vegetation', 'classval': 4, 'is_habitat': True}
for j in range(119, 125):
    LANDCLASSES[j] = {'name': 'wetland short vegetation', 'classval': 5, 'is_habitat': True}
for j in range(125, 149):
    LANDCLASSES[j] = {'name': 'weland tree cover', 'classval': 6, 'is_habitat': True}
for j in range(200, 208):
    LANDCLASSES[j] = {'name': 'open water', 'classval': 7, 'is_habitat': True}
LANDCLASSES[241] = {'name': 'snow and ice', 'classval': 8, 'is_habitat': False}
LANDCLASSES[241] = {'name': 'cropland', 'classval': 9, 'is_habitat': False}
LANDCLASSES[250] = {'name': 'built up', 'classval': 10, 'is_habitat': False}
LANDCLASSES[254] = {'name': 'ocean', 'classval': 11, 'is_habitat': False}

for j in range(256):
    if not j in list(LANDCLASSES.keys()):
        LANDCLASSES[j] = {'name': 'nodata', 'classval': 0, 'is_habitat': False}
        
froms = list(range(256))
tos = [LANDCLASSES[j]['classval'] for j in froms]        
LCLUC2000 = ee.Image('projects/glad/GLCLU2020/LCLUC_2000').remap(froms, tos, 0)
LCLUC2020 = ee.Image('projects/glad/GLCLU2020/LCLUC_2020').remap(froms, tos, 0)

In [ ]:
#Convert to habitat/nonhabitat
froms = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
tos = [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
habitat_2000 = LCLUC2000.remap(froms, tos, 0)
habitat_2020 = LCLUC2020.remap(froms, tos, 0)

In [ ]:
new_habitat = habitat_2000.eq(0).multiply(habitat_2020)
new_habitat_types = LCLUC2020.updateMask(new_habitat)

In [ ]:
habitat2020_types = LCLUC2020.updateMask(habitat_2020)

In [ ]:
def get_ind(fc):
    count_new_habitat_types = new_habitat_types.reduceRegions(**{
        'reducer': ee.Reducer.countDistinctNonNull(),
        'collection': fc,
        'scale': 30
    })
    count_2020_habitat_types = habitat2020_types.reduceRegions(**{
        'reducer': ee.Reducer.countDistinctNonNull(),
        'collection': fc,
        'scale': 30
    })
    ind = (geemap.ee_to_pandas(count_new_habitat_types)['count']) / (geemap.ee_to_pandas(count_2020_habitat_types)['count'])
    return ind

def get_count(fc):
    count_new_habitat_types = new_habitat_types.reduceRegions(**{
        'reducer': ee.Reducer.countDistinctNonNull(),
        'collection': fc,
        'scale': 30
    })
    return (geemap.ee_to_pandas(count_new_habitat_types)['count'])

def get_total(fc):
    count_2020_habitat_types = habitat2020_types.reduceRegions(**{
        'reducer': ee.Reducer.countDistinctNonNull(),
        'collection': fc,
        'scale': 30
    })
    return (geemap.ee_to_pandas(count_2020_habitat_types)['count'])

In [ ]:
all_results = []
for i in range(0,len(boundary_georef)):
    for boundary_name in ['aoi_boundary_name', 'units_boundary_name']:
        if type(boundary_georef.loc[i, boundary_name]) != float: # sometimes boundary_id is nan
            boundary_id = boundary_georef.loc[i, 'geo_name']+'-' + boundary_georef.loc[i, boundary_name]
            print(boundary_id)
            boundary_path = aws_s3_dir + boundary_ext +'boundary-'+boundary_id+'.geojson'
            boundary_geo = requests.get(boundary_path).json()
            temp_gdf = gpd.GeoDataFrame.from_features(boundary_geo)
                        
            boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
            temp_gdf = gpd.GeoDataFrame.from_features(boundary_geo)
            #if True or boundary_name == 'aoi_boundary_name':  # Calculates naturalarea for aoi, uses it for all unit-of-analysis calculations.
               
            temp_gdf['LND_5_numberofHabitatTypesRestoredby2020'] = get_ind(boundary_geo_ee).fillna(-9999)
            all_results.append(temp_gdf.copy())
            outp = pd.concat(all_results, axis=0)[['geo_id', 'geo_level', 'geo_name', 'geo_parent_name', 'LND_5_numberofHabitatTypesRestoredby2020']]
            outp.to_csv(OUTPUT_FILENAME)
    

In [ ]:
processedcities = pd.read_csv(OUTPUT_FILENAME)
processedcities

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir +'/'+ indicators_file_aws)
cities_indicators

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    else:
        print("add new indicators")
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = processedcities,
                                            indicator_name = 'LND_5_numberofHabitatTypesRestoredby2020')

In [ ]:
cities_indicators_merged

## Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = indicators_file_aws
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')